In [1]:
import sys
import re
import os
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import subprocess

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
epochs = 80
trainable = True

In [3]:
from keras_contrib.layers import CRF


In [4]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [5]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [6]:

# preproecessing data from GermEval
def get_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences



In [7]:
train_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-train.tsv')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-dev.tsv')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-test.tsv')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


23999
2199
5099


In [8]:
print(train_sentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [9]:
glove_words=[]

with open('/home/jindal/notebooks/embeddings/embeddings/model_GloVe_w2.embeddings','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
        glove_words.append(word)

In [11]:
print(glove_words[0])

.


In [12]:
print(train_sentences[100])

[['Der', 'O'], ['Meteoritensucher', 'O'], ['hatte', 'O'], ['im', 'O'], ['Frühjahr', 'O'], ['auf', 'O'], ['der', 'O'], ['dänischen', 'B-LOCderiv'], ['Insel', 'O'], ['Lolland', 'B-LOC'], ['30', 'O'], ['Gramm', 'O'], ['eines', 'O'], ['Meteoriten', 'O'], ['aufgespürt', 'O'], [',', 'O'], ['der', 'O'], ['Mitte', 'O'], ['Januar', 'O'], ['über', 'O'], ['Nordeuropa', 'B-LOC'], ['beobachtet', 'O'], ['worden', 'O'], ['war', 'O'], ['.', 'O']]


In [13]:
# train_sentences = modify_labels(train_sentences)
# dev_sentences = modify_labels(dev_sentences)
# test_sentences = modify_labels(test_sentences)

In [13]:
trainSentences = addCharInformatioin(train_sentences)
devSentences = addCharInformatioin(dev_sentences)
testSentences = addCharInformatioin(test_sentences)


In [14]:
print(glove_words[:10])

['.', ',', 'der', 'und', '####', 'die', 'in', '"', '-', '##']


In [15]:
print(len(glove_words))

373861


In [19]:
'##' in glove_words


True

In [20]:
test_sentences_iv=[]
test_sentences_oov=[]

for sentence in testSentences:
    print()
    flag=False
    for word in sentence:
        token=word[0]
        token=re.sub('\d','#',token)
#         print(token)
        if token not in glove_words:
            print('no: %s' %token)
            flag=True
            break
        else:
            print('yes: %s' %token)
    if flag:
        test_sentences_oov.append(sentence)
    else:
        test_sentences_iv.append(sentence)


yes: ####
yes: bis
yes: ####
yes: wurde
yes: der
yes: nördliche
yes: Teil
yes: als
yes: Jugendburg
yes: des
yes: Kolpingwerkes
yes: gebaut
yes: .

yes: Da
yes: Muck
yes: das
no: Kriegsschreiben

yes: Mit
no: #.

yes: Die
yes: These
yes: ,
yes: Schlatter
yes: sei
yes: Antisemit
yes: gewesen
yes: ,
yes: wurde
yes: seither
yes: in
yes: der
yes: theologischen
yes: Fachliteratur
yes: nicht
yes: mehr
yes: vertreten
yes: .

yes: "
yes: Lehmbruck
yes: -
yes: Beuys
no: .Zeichnungen

yes: Der
no: Videohoster

yes: Die
yes: Kanzel
yes: befindet
yes: sich
yes: an
yes: der
yes: Südseite
yes: des
yes: Saals
yes: .

yes: Zuletzt
yes: war
yes: es
yes: Wolfram
no: Graf-Rudolf

yes: Nach
yes: den
yes: Symptomen
yes: lassen
yes: sich
yes: die
yes: beiden
yes: Krankheiten
yes: -
yes: "
yes: normale
yes: "
yes: Influenza
yes: und
yes: Schweinegrippe
yes: -
yes: nicht
yes: unterscheiden
yes: .

yes: Politisch
yes: interessiert
yes: ging
no: Antommarchi

yes: Am
no: ##.

yes: Das
yes: Auto
yes: geriet
yes: 

yes: war
yes: die
yes: sogenannte
no: Goldmaske

yes: Um
no: ##:##

yes: Er
yes: liebt
yes: alles
yes: ,
yes: was
yes: mit
yes: Kunst
yes: zu
yes: tun
yes: hat
yes: .

yes: Selbst
yes: wenn
yes: sein
yes: Leben
yes: unstet
yes: und
yes: wenig
yes: ortsgebunden
yes: ist
yes: ,
yes: so
yes: ist
yes: es
yes: doch
yes: alles
yes: andere
yes: als
yes: ungeordnet
yes: und
no: zügellos

yes: Karlsruhe
yes: -
yes: Das
yes: Bundesverfassungsgericht
yes: wird
yes: sein
yes: Urteil
yes: zur
yes: vorgezogenen
yes: Bundestagswahl
yes: in
yes: der
yes: Woche
yes: ab
yes: dem
no: ##.

yes: Die
no: schmal-lanzettlichen

yes: Das
yes: Ziel
yes: sei
yes: bescheiden
yes: ,
yes: aber
yes: realistisch
yes: ,
yes: schreibt
yes: die
yes: Mailänder
yes: Tageszeitung
yes: "
yes: Corriere
yes: della
yes: Sera
yes: "
yes: .

yes: Beide
yes: Fahrgestelle
yes: waren
yes: auch
yes: nur
yes: mit
yes: Mechanik
yes: und
yes: Motorverkleidung
yes: (
yes: Chassis
yes: &
yes: Cowl
yes: )
yes: erhältlich
yes: .

yes: Ich


no: ##.

yes: Mit
yes: den
yes: Fachstellen
yes: wird
yes: genügend
yes: getan
yes: ,
yes: um
yes: die
yes: Tiere
yes: besser
yes: vor
yes: Quälereien
yes: und
yes: falscher
yes: Haltung
yes: zu
yes: schützen
yes: ?

yes: Seine
yes: Natur
yes: und
no: Entstehungsart

yes: Jolie
yes: und
yes: Pitt
yes: haben
yes: bereits
yes: drei
yes: Kinder
yes: :
yes: den
yes: fünf
yes: Jahre
yes: alten
yes: Maddox
yes: aus
yes: Kambodscha
yes: ,
yes: die
yes: ##
yes: Monate
yes: alte
yes: Zahara
yes: aus
yes: Äthiopien
yes: und
yes: die
yes: sechs
yes: Monate
yes: alte
yes: Shiloh
yes: Nouvel
yes: ,
yes: die
yes: leibliche
yes: Tochter
yes: des
yes: Paares
yes: .

yes: Mariannenplatz
yes: singen
yes: Deutsch
yes: ,
yes: denn
yes: sie
yes: wollen
yes: sich
yes: ausdrücken
yes: .

yes: Haribo
yes: war
yes: für
yes: eine
yes: Stellungnahme
yes: am
yes: Montag
yes: nicht
yes: zu
yes: erreichen
yes: .

yes: Zunächst
yes: beginnt
yes: man
yes: damit
yes: seinen
yes: Charakter
yes: zu
yes: trainieren
yes: 

yes: Präparierung
yes: bestand
yes: und
yes: wo
yes: an
yes: den
yes: Saiten
yes: man
yes: sie
yes: befestigte
yes: .

yes: Nach
yes: dem
yes: Motto
yes: „
yes: Natur
yes: Natur
yes: sein
yes: lassen
yes: “
yes: soll
yes: sich
yes: hier
yes: die
yes: Wildnis
yes: von
yes: morgen
yes: entwickeln
yes: .

yes: AW
yes: :
yes: News
yes: -
no: pcaction.de-Shop

yes: Warwick
yes: erreichte
yes: damit
yes: jeweils
yes: das
yes: Ziel
yes: ,
yes: blieb
yes: aber
yes: außerhalb
yes: der
yes: Punkteränge
yes: .

yes: Drei
yes: Tage
yes: später
yes: griff
yes: das
no: U-Boot

yes: Jedes
no: REA-Team

yes: Während
yes: der
yes: operative
yes: Gewinn
yes: der
no: Mobilfunksparte

yes: In
yes: einer
yes: geschlossenen
yes: Halle
yes: mit
yes: tausenden
yes: begeisterter
yes: Fans
yes: drohen
yes: gesundheitliche
yes: Schäden
yes: .

yes: Während
yes: der
yes: Nazizeit
yes: blieben
yes: politische
yes: Äußerungen
yes: die
yes: Ausnahme
yes: ;
yes: der
yes: Verleger
yes: hielt
yes: sich
yes: weitgehend


yes: Bei
yes: kleineren
yes: Installationen
yes: ist
yes: der
no: snoopingbasierte

yes: Aufgrund
yes: seiner
yes: Berühmtheit
yes: wurde
yes: er
yes: in
yes: die
yes: Briefmarkenserie
yes: „
yes: Leuchttürme
yes: “
yes: der
yes: Deutschen
yes: Post
yes: AG
yes: aufgenommen
yes: .

yes: In
yes: dem
no: Schlussprint

yes: Digitale
yes: Bibliothek
yes: ###
yes: ,
yes: Directmedia
yes: Publishing
yes: Berlin
yes: (
yes: ISBN
no: #-#####-###-#

yes: Gewertet
yes: werden
yes: entsprechend
yes: auch
no: Teamzeiten

yes: Unreal
yes: Tournament
yes: wurde
yes: ####
yes: nicht
yes: mehr
yes: gespielt
yes: ,
yes: dafür
yes: feierten
yes: Dawn
yes: of
yes: War
yes: und
yes: Dead
yes: or
yes: Alive
yes: ihre
yes: Premiere
yes: auf
yes: den
yes: WCG
yes: .

yes: Heute
yes: befindet
yes: sich
yes: im
yes: Schloss
yes: ein
yes: Museum
yes: mit
yes: einer
yes: umfangreichen
yes: Gemäldegalerie
yes: .

yes: "
yes: Für
yes: uns
yes: als
yes: mittelständisches
yes: Unternehmen
yes: mit
yes: hohem
no: Dat

...

yes: die
yes: stark
yes: gestiegenen
no: Auftragseingänge

yes: Verbindung
yes: zwischen
yes: Himmel
yes: und
yes: Erde
yes: halten
yes: die
yes: "
yes: Sot
yes: Dä
yes: "
yes: ,
yes: die
yes: Masten
yes: für
yes: die
yes: Geister
yes: .

yes: Sie
yes: alle
yes: bieten
yes: den
no: Campteilnehmern

yes: Auch
yes: das
yes: ist
yes: eine
yes: Tatsache
yes: .

yes: Da
yes: sich
yes: nach
yes: der
yes: Regenzeit
yes: sehr
yes: viele
yes: Schwebstoffe
yes: und
yes: somit
yes: auch
no: Beutetierchen

yes: Dabei
yes: zog
yes: sich
yes: der
yes: ältere
no: Rollerfahrer

yes: Mit
yes: seinen
yes: Rundfunksendungen
yes: trug
yes: er
yes: nicht
yes: nur
yes: zur
yes: Popularität
yes: des
yes: deutschen
yes: Rundfunks
yes: bei
yes: ,
yes: sondern
yes: wurde
yes: selbst
yes: ein
yes: gefeierter
yes: Tenor
yes: .

yes: Rice
yes: sei
yes: nicht
yes: auf
yes: Einzelheiten
yes: der
yes: Einsätze
yes: eingegangen
yes: ,
yes: sagte
yes: De
yes: Gucht
yes: laut
yes: der
yes: Nachrichtenagentur
no: Belga



no: PCB-Giftfirma

yes: Ab
yes: Mitte
yes: November
yes: beginnt
yes: für
yes: die
no: Postzusteller

yes: Aber
yes: es
yes: gab
yes: auch
yes: andere
yes: Impulse
yes: .
yes: ####
yes: trat
yes: zum
yes: ersten
yes: Mal
yes: das
no: Writers-in-prison-comittee

yes: Nach
yes: seinen
yes: Quellen
yes: kam
yes: es
yes: im
yes: weltweiten
no: Schamanentum

yes: Es
yes: muss
yes: ja
yes: offensichtlich
yes: weitere
yes: Zugänge
yes: zum
yes: Gelände
yes: geben
yes: ,
yes: denn
yes: nachdem
yes: der
yes: Tunnel
yes: bis
yes: in
yes: die
yes: Nacht
yes: hinein
yes: gesperrt
yes: war
yes: ,
yes: sind
yes: die
yes: Besucher
yes: ja
yes: auf
yes: anderem
yes: Weg
yes: irgendwie
yes: vom
yes: Gelände
yes: gekommen
yes: .

yes: Politik
yes: Seit
yes: der
yes: Kommunalwahl
yes: ####
yes: hat
yes: die
yes: Wählergemeinschaft
yes: HWG
yes: zwölf
yes: Sitze
yes: ,
yes: die
yes: CDU
yes: vier
yes: und
yes: die
yes: SPD
yes: drei
yes: Sitze
yes: in
yes: der
yes: Gemeindevertretung
yes: .

yes: Im
yes: 

yes: meisten
yes: Arten
yes: bewohnen
no: hocharktische

yes: Wir
yes: müssen
yes: hier
yes: dauerhaften
yes: Frieden
yes: und
yes: Sicherheit
yes: schaffen
yes: .

yes: ####
yes: wurde
yes: ein
yes: erster
yes: kommunaler
yes: Kindergarten
yes: im
yes: Biwak
yes: eröffnet
yes: .

yes: Der
yes: Umsatz
yes: legte
yes: im
yes: Gesamtjahr
yes: um
yes: ein
yes: Prozent
yes: auf
no: ##,#

yes: Seite
yes: kommt
yes: nicht
yes: in
yes: die
no: Top###

no: ©Bild

yes: "
yes: Ich
yes: möchte
yes: erreichen
yes: ,
yes: dass
yes: in
yes: Moscheen
yes: die
yes: religiösen
yes: Unterweisungen
yes: und
yes: die
yes: Predigten
yes: in
yes: deutscher
yes: Sprache
yes: gehalten
yes: werden
yes: "
yes: ,
yes: sagte
yes: er
yes: dem
yes: "
yes: Spiegel
yes: "
yes: .

yes: An
yes: der
yes: Bahnsteigkante
yes: habe
yes: er
yes: gesehen
yes: ,
yes: dass
yes: die
yes: Frau
yes: auf
yes: die
yes: Gleise
yes: vor
yes: den
yes: Zug
yes: fiel
yes: .

yes: Da
yes: ,
yes: gleich
yes: hinter
yes: der
yes: hohen
yes

yes: Es
yes: übernahm
yes: statt
yes: dessen
yes: Leistungen
yes: nach
yes: Norden
yes: und
yes: Süden
yes: .

yes: Mit
yes: der
yes: Konstituierung
yes: der
yes: Landesregierung
yes: Mecklenburgs
yes: am
no: #.

no: ##.

yes: Die
yes: Blätter
yes: müssen
yes: fachgerecht
yes: entsorgt
yes: werden
yes: .

yes: Obwohl
yes: dabei
yes: fast
yes: seine
yes: rechte
yes: Hand
yes: durchtrennt
yes: wurde
yes: und
yes: das
yes: Blut
yes: spritzte
yes: ,
yes: schaffte
yes: er
yes: es
yes: noch
yes: ,
yes: die
yes: Maschine
yes: abzustellen
yes: und
yes: sich
yes: einem
yes: einfahrenden
yes: Lkw
yes: entgegenzustellen
yes: ,
yes: um
yes: auf
yes: seine
yes: Verletzung
yes: aufmerksam
yes: zu
yes: machen
yes: .

yes: Aufgrund
yes: seiner
yes: politischen
yes: Überzeugungen
yes: und
yes: Aktivitäten
yes: legte
yes: das
yes: FBI
yes: eine
yes: Akte
yes: von
yes: über
no: ##.###

yes: Der
yes: Grund
yes: für
yes: diese
yes: Verurteilung
yes: war
yes: seine
yes: seit
yes: vielen
yes: Jahren
yes: bes

yes: ihm
yes: das
yes: durchgehen
yes: zu
yes: lassen
yes: .

yes: Wer
yes: will
yes: ,
yes: dass
yes: von
yes: den
yes: hiesigen
yes: Instituten
yes: wenigstens
yes: die
yes: Deutsche
yes: Bank
yes: dabei
yes: als
yes: Akteur
yes: auftritt
yes: und
yes: nicht
yes: zum
yes: Opfer
yes: wird
yes: ,
yes: muss
yes: ihren
yes: Kurs
yes: akzeptieren
yes: ,
yes: auch
yes: wenn
yes: das
yes: schwer
yes: fällt
yes: .

yes: Alle
yes: Inhalte
yes: der
yes: Pressemappe
yes: anzeigen
yes: Horst
no: Jaedicke

yes: Bei
no: Reichertsheim

yes: Vorkehrungen
yes: zum
yes: Schutz
yes: der
yes: Privatsphäre
yes: informiert
yes: werden
yes: .

yes: ####
yes: wurde
yes: er
yes: amerikanischer
yes: Staatsbürger
yes: .

yes: Eine
yes: erste
yes: wichtige
yes: Etappe
yes: wurde
yes: erreicht
yes: ,
yes: als
yes: die
no: EFTA-Staaten

yes: Ihnen
yes: gehören
yes: etwa
yes: ##
yes: Personen
yes: an
yes: ,
yes: die
yes: sich
yes: auf
yes: sehr
yes: alte
yes: Ausrüstung
yes: verlassen
yes: müssen
yes: .

yes: Fors

In [21]:
print(len(test_sentences_iv))
print(len(test_sentences_oov))
print(len(test_sentences))

2037
3062
5099


In [22]:
labelSet = set()
words = {}
characters= set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token, char, label in sentence:
            for c in char:
                characters.add(c)
            labelSet.add(label)
#             words[token.lower()] = True
            words[re.sub('\d','#',token)] = True

In [23]:
print(labelSet)

{'B-LOCderiv', 'B-PERderiv', 'B-OTHderiv', 'I-LOC', 'B-PERpart', 'I-OTHpart', 'I-OTH', 'B-PER', 'I-LOCpart', 'B-OTHpart', 'B-ORGpart', 'B-ORGderiv', 'I-ORG', 'I-PERderiv', 'B-ORG', 'B-OTH', 'I-OTHderiv', 'B-LOC', 'I-PER', 'B-LOCpart', 'I-ORGpart', 'I-LOCderiv', 'I-ORGderiv', 'O', 'I-PERpart'}


In [24]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [25]:
print(label2Idx)

{'B-LOCderiv': 0, 'B-PERderiv': 1, 'I-LOCderiv': 21, 'I-LOCpart': 8, 'B-ORGpart': 10, 'B-OTHderiv': 2, 'I-LOC': 3, 'I-PERderiv': 13, 'B-ORG': 14, 'B-OTH': 15, 'I-OTHderiv': 16, 'B-LOC': 17, 'I-PER': 18, 'B-LOCpart': 19, 'I-ORGpart': 20, 'B-PERpart': 4, 'I-ORG': 12, 'I-OTHpart': 5, 'I-ORGderiv': 22, 'I-OTH': 6, 'O': 23, 'B-ORGderiv': 11, 'B-PER': 7, 'I-PERpart': 24, 'B-OTHpart': 9}


In [26]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [27]:
string_words = ' '.join(words.keys())
# print(string_words)
f = open('/home/jindal/notebooks/fastText/german_words.txt','wb')
f.write(string_words.encode())
f.close()

In [28]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'mainly_numeric': 5, 'initialUpper': 3, 'allUpper': 2, 'allLower': 1, 'numeric': 0}


In [29]:
word2Idx={}
wordEmbeddings=[]

fEmbeddings = open("/home/jindal/notebooks/embeddings/embeddings/model_GloVe_w2.embeddings", encoding="utf-8")

for line in fEmbeddings:
    split = line.strip().split(" ")
    word = split[0]
#     print(word)
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(len(split)-1) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)
        
        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, len(split)-1)
        wordEmbeddings.append(vector)

    if split[0] in words:
        vector = np.array([float(num) for num in split[1:]])
        wordEmbeddings.append(vector)
        word2Idx[word] = len(word2Idx)
wordEmbeddings=np.array(wordEmbeddings)

In [30]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings))
print(len(wordEmbeddings[2]))

60615
300


In [31]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{'¤': 0, 'Ż': 1, '≤': 2, 'ž': 3, 'ú': 4, 'в': 5, '\x99': 308, '柯': 7, 'Œ': 8, 'τ': 10, 'У': 325, '동': 11, 'а': 13, 'Ш': 14, 'ř': 15, 'ʻ': 16, 'أ': 17, '8': 18, 'ı': 172, 'C': 20, '‚': 21, '傳': 24, ',': 25, '\x92': 26, '2': 79, 'D': 29, '§': 278, '→': 30, 'ĩ': 31, 'í': 32, 'õ': 33, 'я': 318, 'ā': 34, '†': 35, '佐': 36, 'H': 37, '-': 38, 'ŏ': 220, 'ῦ': 39, 'к': 40, '_': 43, 'R': 42, 'ó': 218, 'Q': 213, '–': 44, 'a': 45, 'v': 46, 'х': 289, 'ċ': 225, 'Ä': 47, 'œ': 61, 'á': 48, 'С': 6, '·': 53, '©': 52, 'ю': 54, 'G': 55, 'e': 57, 'b': 58, 'ğ': 59, '«': 60, 'η': 192, ';': 62, '.': 63, 'ū': 64, 'ě': 65, 'ă': 66, '台': 262, 'ź': 67, 'Þ': 68, 'u': 69, '(': 70, '€': 71, 'б': 72, '冲': 73, 'r': 74, 'ß': 75, '\x94': 78, '´': 77, 'ō': 80, 'À': 242, 'm': 81, 'オ': 250, '¸': 83, '‐': 283, '≘': 85, '李': 23, 'ɨ': 190, 'Λ': 86, 'Л': 82, 'н': 90, '南': 93, '\x95': 91, '″': 92, 'ḳ': 95, 'ş': 96, 'ы': 97, 'x': 98, '=': 99, 'М': 100, 'φ': 219, 'ś': 102, '別': 103, 'е': 104, '/': 106, 'κ': 231, 'y': 108, 'P': 109,

In [32]:
print(trainSentences[0]) 

[['Schartau', ['S', 'c', 'h', 'a', 'r', 't', 'a', 'u'], 'B-PER'], ['sagte', ['s', 'a', 'g', 't', 'e'], 'O'], ['dem', ['d', 'e', 'm'], 'O'], ['"', ['"'], 'O'], ['Tagesspiegel', ['T', 'a', 'g', 'e', 's', 's', 'p', 'i', 'e', 'g', 'e', 'l'], 'B-ORG'], ['"', ['"'], 'O'], ['vom', ['v', 'o', 'm'], 'O'], ['Freitag', ['F', 'r', 'e', 'i', 't', 'a', 'g'], 'O'], [',', [','], 'O'], ['Fischer', ['F', 'i', 's', 'c', 'h', 'e', 'r'], 'B-PER'], ['sei', ['s', 'e', 'i'], 'O'], ['"', ['"'], 'O'], ['in', ['i', 'n'], 'O'], ['einer', ['e', 'i', 'n', 'e', 'r'], 'O'], ['Weise', ['W', 'e', 'i', 's', 'e'], 'O'], ['aufgetreten', ['a', 'u', 'f', 'g', 'e', 't', 'r', 'e', 't', 'e', 'n'], 'O'], [',', [','], 'O'], ['die', ['d', 'i', 'e'], 'O'], ['alles', ['a', 'l', 'l', 'e', 's'], 'O'], ['andere', ['a', 'n', 'd', 'e', 'r', 'e'], 'O'], ['als', ['a', 'l', 's'], 'O'], ['überzeugend', ['ü', 'b', 'e', 'r', 'z', 'e', 'u', 'g', 'e', 'n', 'd'], 'O'], ['war', ['w', 'a', 'r'], 'O'], ['"', ['"'], 'O'], ['.', ['.'], 'O']]


In [34]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences, word2Idx,  label2Idx, case2Idx, char2Idx))
dev_set = padding(createMatrices(devSentences, word2Idx, label2Idx, case2Idx, char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx, char2Idx))
test_set_iv= padding(createMatrices(test_sentences_iv, word2Idx, label2Idx, case2Idx, char2Idx))
test_set_oov = padding(createMatrices(test_sentences_oov, word2Idx, label2Idx, case2Idx, char2Idx))

In [35]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
print((train_set[0][2]))

[[56253, 2573, 25, 9, 21305, 9, 79, 6778, 3, 2272, 275, 9, 8, 47, 1053, 10801, 3, 7, 872, 234, 28, 16148, 35, 9, 2], [3, 1, 1, 4, 3, 4, 1, 3, 4, 3, 1, 4, 1, 1, 3, 1, 4, 1, 1, 1, 1, 1, 1, 4, 4], array([[267, 264,  12, ...,   0,   0,   0],
       [148,  45, 167, ...,   0,   0,   0],
       [287,  57,  81, ...,   0,   0,   0],
       ...,
       [310,  45,  74, ...,   0,   0,   0],
       [ 41,   0,   0, ...,   0,   0,   0],
       [ 63,   0,   0, ...,   0,   0,   0]], dtype=int32), [7, 23, 23, 23, 14, 23, 23, 23, 23, 7, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23]]
25
[[267 264  12 ...   0   0   0]
 [148  45 167 ...   0   0   0]
 [287  57  81 ...   0   0   0]
 ...
 [310  45  74 ...   0   0   0]
 [ 41   0   0 ...   0   0   0]
 [ 63   0   0 ...   0   0   0]]


In [36]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [37]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [38]:
print(test_sentences_iv[0])

[['1951', ['1', '9', '5', '1'], 'O'], ['bis', ['b', 'i', 's'], 'O'], ['1953', ['1', '9', '5', '3'], 'O'], ['wurde', ['w', 'u', 'r', 'd', 'e'], 'O'], ['der', ['d', 'e', 'r'], 'O'], ['nördliche', ['n', 'ö', 'r', 'd', 'l', 'i', 'c', 'h', 'e'], 'O'], ['Teil', ['T', 'e', 'i', 'l'], 'O'], ['als', ['a', 'l', 's'], 'O'], ['Jugendburg', ['J', 'u', 'g', 'e', 'n', 'd', 'b', 'u', 'r', 'g'], 'O'], ['des', ['d', 'e', 's'], 'O'], ['Kolpingwerkes', ['K', 'o', 'l', 'p', 'i', 'n', 'g', 'w', 'e', 'r', 'k', 'e', 's'], 'B-OTH'], ['gebaut', ['g', 'e', 'b', 'a', 'u', 't'], 'O'], ['.', ['.'], 'O']]


In [39]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5, name='dropout1')(embed_char_out)

# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
# output = concatenate([words, casing, char])

kernel_sizes = (3, 4, 5)
conv_blocks = []
for sz in kernel_sizes:
    conv = TimeDistributed(Conv1D(
                         kernel_size=sz,
                         filters=32,
                         padding="same",
                         activation="relu",
                         strides=1))(embed_char_out)
    conv = TimeDistributed(MaxPooling1D(52))(conv)
    conv = TimeDistributed(Flatten())(conv)
#     conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
output = concatenate([words, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])

output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
crf = CRF(len(label2Idx))
output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
# model.compile(loss=crf.loss_function, optimizer=Adam(lr = 0.002), metrics=[crf.accuracy])
model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10496       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_4 (TimeDistrib (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [ ]:
for x in range(1):
#     print(i)
    maxf1 = 0
    for epoch in range(epochs):    
        print("Epoch %d/%d"%(epoch, epochs))
        a = Progbar(len(train_batch_len))
        for i, batch in enumerate(iterate_minibatches(train_batch, train_batch_len)):
            labels, tokens, casing, char = batch
            labels = labels.tolist()
    #         print(labels)
            for sentence in labels:
                for i,label in enumerate(sentence):
                    temp = [0]*len(label2Idx)
                    value = label[0]
                    temp[value]=1
                    sentence[i] = temp
            labels = np.array(labels)
    #         print(labels)
            model.train_on_batch([tokens, casing, char], labels)
            a.update(i)
        if epoch >= 50:
            predLabels, correctLabels = tag_dataset(dev_batch)        
            pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
#             predLabels = np.concatenate(predLabels).ravel()
#             correctLabels = np.concatenate(correctLabels).ravel()
#             f1_dev = sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' )
            print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))
#             print(f1_dev)

            if f1_dev > maxf1:
                model.save('german_ner_oov_glove_w2.h5')
                maxf1 = f1_dev

    id2word = {v: k for k, v in word2Idx.items()}
    print(len(correctLabels))        

    

Epoch 0/80
55/51 [================================] - 40s 724ms/step
Epoch 1/80
55/51 [================================] - 13s 233ms/step
Epoch 2/80
55/51 [================================] - 12s 212ms/step
Epoch 3/80
55/51 [================================] - 12s 214ms/step
Epoch 4/80
55/51 [================================] - 12s 215ms/step
Epoch 5/80
55/51 [================================] - 12s 212ms/step
Epoch 6/80
55/51 [================================] - 13s 233ms/step
Epoch 7/80
55/51 [================================] - 12s 221ms/step
Epoch 8/80
55/51 [================================] - 11s 204ms/step
Epoch 9/80
55/51 [================================] - 12s 211ms/step
Epoch 10/80
55/51 [================================] - 12s 219ms/step
Epoch 11/80
55/51 [================================] - 12s 224ms/step
Epoch 12/80
55/51 [================================] - 12s 218ms/step
Epoch 13/80
55/51 [================================] - 12s 215ms/step
Epoch 14/80
55/51 [===========

2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.755, Rec: 0.776, F1: 0.765
Epoch 59/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.791, Rec: 0.777, F1: 0.784
Epoch 60/80
1843/2199 [========================>.....] - ETA: 7s

In [ ]:
model.load_weights('german_ner_oov_glove_w2.h5')
test_batch_oov, y = createBatches(test_set_oov)
predLabels, correctLabels = tag_dataset(test_batch_oov)        
pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

id = 1
file_name='test_oov_glove_w2'+'.txt'
input_file_name = '/home/jindal/notebooks/jindal/NER/'+file_name
f = open(input_file_name,'wb')
for sentence_number, batch in enumerate(test_batch_oov):
    for word_number, wordid in enumerate(batch[0]):
        word = id2word[wordid]
        true_label = correctLabels[sentence_number][word_number]
        true_label = idx2Label[true_label]
        pred_label = predLabels[sentence_number][word_number]
        pred_label = idx2Label[pred_label]

        string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
#             string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
        string = string.encode()
        f.write(string)
        id+=1

command = "perl nereval.perl"
directory_germeval = r'/home/jindal/notebooks/Resources/GermEVAL'
#     directory_conll = r'/home/jindal/notebooks/Resources/CONLL/2003/ner/bin'
#     command = "perl conlleval"

output_file_name = '/home/jindal/notebooks/jindal/NER/test_result_oov_glove_w2'+'.txt'
output_file=open(output_file_name,'w')
input_file = open(input_file_name)
import subprocess
process = subprocess.Popen(command.split(), stdin=input_file, stdout=output_file, cwd=directory_germeval)
out, err = process.communicate()
#     print(out)
print('******************************************************************************')

In [ ]:
model.load_weights('german_ner_oov_glove_w2.h5')
test_batch_iv , x =  createBatches(test_set_iv)
predLabels, correctLabels = tag_dataset(test_batch_iv)        
pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

id = 1
file_name='test_iv_glove_w2'+'.txt'
input_file_name = '/home/jindal/notebooks/jindal/NER/'+file_name
f = open(input_file_name,'wb')
for sentence_number, batch in enumerate(test_batch_iv):
    for word_number, wordid in enumerate(batch[0]):
        word = id2word[wordid]
        true_label = correctLabels[sentence_number][word_number]
        true_label = idx2Label[true_label]
        pred_label = predLabels[sentence_number][word_number]
        pred_label = idx2Label[pred_label]

        string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
#             string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
        string = string.encode()
        f.write(string)
        id+=1

command = "perl nereval.perl"
directory_germeval = r'/home/jindal/notebooks/Resources/GermEVAL'
#     directory_conll = r'/home/jindal/notebooks/Resources/CONLL/2003/ner/bin'
#     command = "perl conlleval"

output_file_name = '/home/jindal/notebooks/jindal/NER/test_result_iv_glove_w2'+'.txt'
output_file=open(output_file_name,'w')
input_file = open(input_file_name)
import subprocess
process = subprocess.Popen(command.split(), stdin=input_file, stdout=output_file, cwd=directory_germeval)
out, err = process.communicate()
#     print(out)
print('******************************************************************************')